---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import re
def clean_region(text):
    text = re.sub("\[.*\]","",text)
    text = re.sub("\(.*","",text)
    text = text.rstrip()
    return text

def clean_state(state):
    state = re.sub("\[.*\]","",state)
    state = state.rstrip()
    return state

    
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = []
    state = None
    with open('university_towns.txt') as file:
        for line in file:
            if line.find("[edit]") > -1:
                state = clean_state(line)
            else:
                region = clean_region(line)
                university_towns.append((state,region))
    df = pd.DataFrame(university_towns,columns=['State','RegionName'])
    return df


In [4]:
def set_change(row):
    if row['GDP'] >= row['GDP'].shift(-1):
        row['GDP_CHANGE'] = 'INCREASE'
    else:
        row['GDP_CHANGE'] = 'DECREASE'
    return row
    

gd_plev = pd.read_excel('gdplev.xls',skiprows=[0,1,2,3,4,5,6])
gd_plev.rename(columns={'Unnamed: 4':'Quarter','Unnamed: 6':'GDP'},inplace=True)

gd_plev = gd_plev[['Quarter','GDP']]

gd_plev = gd_plev.iloc[212:]
gd_plev.reset_index(inplace=True)
gd_plev['GDP'].astype(float)
gd_plev = gd_plev[['Quarter','GDP']]

gd_plev['GDP_CHANGE'] = ''


for i in range(1,len(gd_plev)):
    prev_value = gd_plev.iloc[i-1]['GDP']
    new_value = gd_plev.iloc[i]['GDP']
    if new_value >= prev_value:
        gd_plev.iloc[i,2] = "INCREASE"
    else:
        gd_plev.iloc[i,2] = "DECREASE"
# print(gd_plev.head())

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    state = None
    for i in range(1,len(gd_plev)-4):
        current = gd_plev.iloc[i,2]
        current_1 = gd_plev.iloc[i+1,2]
        if current == "DECREASE" and current_1 == "DECREASE":
            state = gd_plev.iloc[i,0]
            break
    return state

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    state = None
    start = get_recession_start()
    start = gd_plev[gd_plev['Quarter']==start].index[0]
    for i in range(start,len(gd_plev)-2):
        current = gd_plev.iloc[i,2]
        current_1 = gd_plev.iloc[i+1,2]
        if current == "INCREASE" and current_1 == "INCREASE":
            state = gd_plev.iloc[i+1,0]
            break
    return state

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    index_start = gd_plev[gd_plev['Quarter']==start].index[0]
    end = get_recession_end()
    index_end = gd_plev[gd_plev['Quarter']==end].index[0]
    min = gd_plev.iloc[index_start]['GDP']
    min_index = index_start
    for i in range(index_start+1,index_end+1):
        gdp = gd_plev.iloc[i]['GDP']
        if gdp < min:
            min = gdp
            min_index = i
    return gd_plev.iloc[min_index,0]
get_recession_bottom()

'2009q2'

In [13]:
def generate_dict(housing_data):
    dict = {}
    start_index = housing_data.columns.get_loc('2000-01')
    columns = housing_data.columns
    for year in range(2000,2016):
        for q in range(1,5):
            key = str(year)+"q"+str(q)
            arr = []
            arr.append(columns[start_index])
            arr.append(columns[start_index+1])
            arr.append(columns[start_index+2])
            start_index +=3
            dict[key]=arr
    dict['2016q1']=['2016-01','2016-02','2016-03']
    dict['2016q2']=['2016-04','2016-05','2016-06']
    dict['2016q3']=['2016-07','2016-08']
    return dict
def change_housing_state(row):
    if row['State'] in states:
        row['State'] = states[row['State']]
    return row
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1,inplace=True)
    start_index = housing_data.columns.get_loc('1996-04')
    end_index = housing_data.columns.get_loc('1999-12')
#     print(str(start_index) + " " + str(end_index))
    delete_list = []
    for i in range(start_index,end_index+1):
        delete_list.append(i)
    housing_data.drop(housing_data.columns[delete_list],axis=1,inplace=True)
    column_dict = generate_dict(housing_data)
    housing_data = housing_data.apply(change_housing_state,axis=1)
    for key,value in column_dict.items():
        housing_data[key] = housing_data[column_dict[key]].mean(axis=1)
        housing_data.drop(column_dict[key],axis=1,inplace=True)
    housing_data = housing_data.set_index(['State','RegionName'])
    return housing_data
# convert_housing_data_to_quarters()

In [35]:
import scipy.stats as stats
def get_quarters():
    arr = []
    for year in range(2000,2016):
        for q in range(1,5):
            key = str(year)+"q"+str(q)
            arr.append(key)
    arr.append("2016q1")
    arr.append("2016q2")
    arr.append("2016q3")
    return arr
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    housing_data = convert_housing_data_to_quarters()
    universities = get_list_of_university_towns().to_records(index=False).tolist()
    recession_start = '2008q2'
    recession_bottom = get_recession_bottom()
    housing_data['Price_Ratio'] = housing_data[recession_start].div(housing_data[recession_bottom])
    housing_data_university = housing_data.loc[universities]
    housing_data_nu = housing_data.loc[~housing_data.index.isin(universities)]
    mean_u = housing_data_university['Price_Ratio'].mean()
    mean_nu = housing_data_nu['Price_Ratio'].mean()
    better = None
    if mean_u<mean_nu:
        better = "university town"
    else:
        better = "non-university town"
    statistics, pValue = stats.ttest_ind(housing_data_university['Price_Ratio'], housing_data_nu['Price_Ratio'],nan_policy='omit')
    if pValue <0.01:
        return (True,pValue,better)
    else:
        return (False,pValue,better)
type(run_ttest())

tuple